# 2.b Data Preprocessing and Feature Engineering (Sliding Window (Open))

### Import necessary Packages

In [369]:
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import boto3 
import io  
from sagemaker import get_execution_role
role = get_execution_role()
# Library to suppress warnings or deprecation notes
import warnings
warnings.filterwarnings("ignore")

In [370]:
session = boto3.Session()

In [371]:
role

'arn:aws:iam::190183093544:role/service-role/AmazonSageMaker-ExecutionRole-20220602T141236'

### Getting Data from Yahoo Finance¶

In [372]:
btc = f'https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=1410912000&period2=1656633600&interval=1d&events=history&includeAdjustedClose=true'

### Load the Data

In [373]:
data= pd.read_csv(btc)


### Making new dataframe with requried features

In [374]:
data = data[['Open']]

In [375]:
data

,Open
0,465.864014
1,456.859985
2,424.102997
3,394.673004
4,408.084991
...,...
2840,21028.238281
2841,20731.544922
2842,20281.169922
2843,20108.312500


In [376]:
data.shape

(2845, 1)

In [377]:
data.head()

,Open
0,465.864014
1,456.859985
2,424.102997
3,394.673004
4,408.084991


### Creating a function for sliding window

In [378]:
def mv_window(row, col, i_start_a, win_1, dataset):
    import numpy as np
    X = np.zeros((row, col))
    for d in range(0,win_1):
        for i in range(0,row):
            X[i][d] = dataset['Open'][i+i_start_a+d]
    return X

In [379]:
#selecting the window and size
sliding=mv_window(2815, 30, 0, 30, data)

In [380]:
sliding = pd.DataFrame(data=sliding)

In [381]:
sliding.tail()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
2810,28622.625000,29019.867188,29443.365234,31723.865234,31792.554688,29794.890625,30467.806641,29706.138672,29835.117188,29910.283203,...,20385.718750,20473.425781,19010.902344,20553.371094,20594.294922,20719.414063,19986.607422,21084.648438,21233.609375,21496.494141
2811,29019.867188,29443.365234,31723.865234,31792.554688,29794.890625,30467.806641,29706.138672,29835.117188,29910.283203,31371.742188,...,20473.425781,19010.902344,20553.371094,20594.294922,20719.414063,19986.607422,21084.648438,21233.609375,21496.494141,21028.238281
2812,29443.365234,31723.865234,31792.554688,29794.890625,30467.806641,29706.138672,29835.117188,29910.283203,31371.742188,31151.480469,...,19010.902344,20553.371094,20594.294922,20719.414063,19986.607422,21084.648438,21233.609375,21496.494141,21028.238281,20731.544922
2813,31723.865234,31792.554688,29794.890625,30467.806641,29706.138672,29835.117188,29910.283203,31371.742188,31151.480469,30215.279297,...,20553.371094,20594.294922,20719.414063,19986.607422,21084.648438,21233.609375,21496.494141,21028.238281,20731.544922,20281.169922
2814,31792.554688,29794.890625,30467.806641,29706.138672,29835.117188,29910.283203,31371.742188,31151.480469,30215.279297,30110.330078,...,20594.294922,20719.414063,19986.607422,21084.648438,21233.609375,21496.494141,21028.238281,20731.544922,20281.169922,20108.312500


In [382]:
#since first 30 get ommited
y=data["Open"][30:]

In [383]:
y=pd.DataFrame(y)

### Splitting the last 30 days as the validation set

In [384]:
y_val = y[2785:]

In [385]:
X_val=sliding[2785:]

In [386]:
#last 30 get ommited as they are the validation set
X=sliding[:2785]
y1=y[:2785]

### Split Data to prepare train and test set

In [387]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y1, test_size=0.30, random_state=1
)

In [388]:
print("Shape of Training set : ", X_train.shape)
print("Shape of test set : ", X_test.shape)

Shape of Training set :  (1949, 30)
Shape of test set :  (836, 30)


### Export Data for Model Training

In [389]:
#for method 1
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)
np.save("X_val.npy", X_val)
np.save("y_val.npy", y_val)

In [390]:
s3 = session.resource('s3') 
local_path = 'X_train.npy' 
s3_filename =  'X_train.npy' 
result = s3.Bucket('data-use-case-btc-prediction').upload_file(local_path,s3_filename)  
print(result)

None


In [391]:
s3 = session.resource('s3') 
local_path = 'y_train.npy' 
s3_filename =  'y_train.npy' 
result = s3.Bucket('data-use-case-btc-prediction').upload_file(local_path,s3_filename)  
print(result)

None


In [392]:
s3 = session.resource('s3') 
local_path = 'X_test.npy' 
s3_filename =  'X_test.npy' 
result = s3.Bucket('data-use-case-btc-prediction').upload_file(local_path,s3_filename)  
print(result)

None


In [393]:
s3 = session.resource('s3') 
local_path = 'y_test.npy' 
s3_filename =  'y_test.npy' 
result = s3.Bucket('data-use-case-btc-prediction').upload_file(local_path,s3_filename)  
print(result)

None


In [394]:
s3 = session.resource('s3') 
local_path = 'X_val.npy' 
s3_filename =  'X_val.npy' 
result = s3.Bucket('data-use-case-btc-prediction').upload_file(local_path,s3_filename)  
print(result)

None


In [395]:
s3 = session.resource('s3') 
local_path = 'y_val.npy' 
s3_filename =  'y_val.npy' 
result = s3.Bucket('data-use-case-btc-prediction').upload_file(local_path,s3_filename)  
print(result)

None
